# <center> ConnectX dengan Deep Q-Learning </center>

<div style = "text-align: justify ; line-height: 1.75 em">Kernel ini adalah <b><i>review</b></i> (dengan <b>sedikit perubahan</b> yang menurut saya lebih pas, misal : penggunaan <i>activation function</i>, <i>hyperparameter</i>, aturan decay epsilon, dan aturan pemberian reward) dari kernel yang dibuat oleh <b>Hieu Phung</b> (@phunghieu, <a href = "https://www.kaggle.com/phunghieu/connectx-with-deep-q-learning-pytorch">ConnectX with Deep Q-Learning (PyTorch)</a>) dan dibuat untuk mempermudah kita memahami prinsip-prinsip <b><i>Reinforcement Learning</b></i> serta bagaimana implementasinya pada sebuah game bernama <b>ConnectX</b> bagi para pemula dalam bidang ini khususnya yang berasal dari Indonesia. Bagi para pembaca silahkan </b><i>up vote</b></i> kernel tersebut. 
<br>
<br>
ConnectX ini adalah game yang bertujuan untuk membuat 4 deret <i>checker</i> (bisa vertikal, horizontal, maupun diagonal) sebelum musuh kita dengan setiap putaran secara bergantian kita dan musuh kita menaruh <i>checker</i> pada papan 6 x 7. Menggunakan penggabungan antara <b>Q-Learning</b> (prinsip dasar <i>Reinforcement Learning</i>) dan <i>Neural Network</i> (<b><i>Deep Learning</b></i>) kita akan melatih sebuah agent untuk bermain dan nantinya akan ditandingkan dengan agent-agent lainnya. Sebagai informasi tambahan, ConnectX adalah kompetisi yang berfokus pada pembelajaran (tanpa <i>medal</i>) sehingga fokus utama kita adalah belajar dari kernel - kernel yang ada dan bukan <i>Leader Board</i> (walaupun untuk submisi per tanggal 28 Januari 2020 agent saya secara kebetulan bisa mengalahkan agent Hieu Phung (peace :))). 
<br>
<br>
Program yang ditulis untuk melatih agent pada kernel ini (dan kebanyakan dalam kasus-kasus penyelesaian <i>Reinforcement Learning</i> lainnya) adalah <i>Object Oriented Programming</i> sehingga sangat disarankan bagi bara pembaca yang terbiasa menggunakan pendekatan <i>Procedural Programming</i> untuk melakukan pengolahan data agar belajar terlebih dahulu tentang <i>class</i>, <i>objects</i>, <i>overloading</i>, dan <i>inheritance</i> di Python.
</div>

## Daftar Isi

1. [Reinforcement Learning dan Q-Learning](#pengertianqlearning)
2. [Prinsip Deep Q-Learning](#deepqlearning)
3. [Implementasi untuk ConnectX](#implementasi)
    + [Tahap Persiapan](#persiapan)
    + [Tahap Pelatihan Agent](#pelatihan)
    + [Menuliskan Agent ke Sebuah File](#menulis)
    + [Melihat Hasil Agent](#hasil)

<a id = "pengertianqlearning"></a>

## Reinforcement Learning dan Q-Learning

<div style = "text-align: justify ; line-height: 1.75 em"><b><i>Reinforcement Learning</b></i> adalah sebuah metode <i>machine learning</i> dimana kita <b>melatih sebuah agent</b> untuk belajar mengambil keputusan <b><i>action</b></i>) dalam sebuah kondisi tertentu (<b><i>state</b></i>) di lingkungan tertentu (<b><i>environment</b></i>) sehingga bisa memaksimalkan <b><i>reward</b></i> yang akan didapat (<i>reward</i> bisa bernilai positif atau negatif dan akan didapat bila berhasil melakukan tujuan yang diinginkan). Proses pembelajaran ini dilakukan pada setiap <b><i>episode</b></i> pembelajaran dimana <b><i>episode</b></i> adalah setiap kejadian diantara <i>initial state</i> (kondisi awal) dan <i>terminal state</i> (kondisi akhir). Pada <i>Reinforcement Learning</i>, diasumsikan bahwa kondisi yang akan datang hanya dipengaruhi oleh kondisi saat ini dan bukan kondisi-kondisi sebelumnya. Proses dengan asumsi seperti ini disebut juga Proses Markov</div>
<br>
<div style = "text-align: justify ; line-height: 1.75 em">Pada setiap prosesnya (setiap <i>state</i>), agent akan melakukan <i>action</i> untuk memaksimalkan <i>reward</i>. Maka kualitas (<b><i>quality</b></i>) dari setiap aksi dinilai dari seberapa besarnya total <i>reward</i> yang didapat. Total <i>reward</i> ini disebut juga dengan <b><i>Q-value</b></i>. Pada tiap <i>episode</i>, agent dilatih untuk berusaha meningkatkan <i>Q-value</i> di <i>state</i> tersebut. Pada setiap pembelajaran, maka akan didapat <i>Q-value</i> akan diperbarui menjadi :</div>

![Proses perbaruan *Q-value*](https://wikimedia.org/api/rest_v1/media/math/render/svg/47fa1e5cf8cf75996a777c11c7b9445dc96d4637)

dengan :

$\alpha$ = porsi seberapa diperhitungkannya pengetahuan baru menggantikan pengetahuan lama <br>
$\gamma$ = porsi seberapa diperhitungkannya reward yang didapat setelah <i>state</i> saat ini (bila 0 maka hanya <i>immadiate reward</i> yang diperhitungkan) <br>

<div style = "text-align: justify ; line-height: 1.75 em">Nantinya <i>Q-value</i> ini akan dimasukkan ke dalam <i>Q-table</i> yang berisi daftar <i>reward</i> untuk setiap <i>action</i> pada setiap <i>state</i>. Untuk lebih jelasnya maka perhatikan ilustrasi di bawah ini (diambil dari <a src = "https://towardsdatascience.com/q-learning-54b841f3f9e4">artikel ini</a>).
<br>

<center><img src = "https://miro.medium.com/max/750/1*tSFotpgBNGurajFg2FH8Cg.png"></center>

Pada sebuah permainan dengan misi mencari jalan terpendek dari 1,1 ke 5,5, terdapat <i>reward</i> yang ditunjukkan dengan warna hijau. Sementara warna merah adalah <i>punishment</i> (<i>reward</i> bernilai negatif sehingga mengurangi <i>Q-value</i>). Maka <i>Q-table</i> yang dihasilkan adalah 
<br>
<br>
<center><img src = "https://miro.medium.com/max/981/1*p6yPonqoDMlK1w_EJKlcAQ.png"></center>
<br>
Artinya, pada titik 1,1, bila kita melakukan aksi bergerak ke atas atau ke kiri kita mendapat <i>punishment</i> sebesar -1000 sementara bila bergerak ke bawah atau ke kanan kita mendapat <i>reward</i> sebesar 1 dan seterusnya. Namun ini baru sistem yang hanya memperhatikan <i>immadiate reward</i> dan sistem ini tidak sempurna. Pada konsisi tertentu, hanya memperhitungkan <i>immadiate reward</i> ($\gamma$ = 0) punya beberapa kelemahan. Misalkan pada suatu <i>environment</i> yang berbeda yaitu permainan yang sama namun dengan suatu halangan, kita ada di titik 2.2
<br>
<center><img src = "https://miro.medium.com/max/750/1*XV1aCvN2kWkTaos-E1h1yQ.png"></center>

Maka terdapat 2 pilihan yaitu mengambil <i>immadiate reward</i> yang besar yaitu ke bawah atau mengambil <i>immadiate reward</i> yang lebih kecil dan keluar dari jalan buntu. Hal lainnya yang perlu diperhatikan adalah agent tersebut belum memperhitungkan banyaknya langkah yang harus diambil untuk memaksimalkan <i>Q-value</i>. Bisa saja untuk memaksimalkan <i>Q-value</i> maka agent kita melakukan <i>infinite looping</i> di daerah berwarna hijau dan mengambil <i>reward</i> sebanyak-banyaknya. Selain itu, bila kita perhatikan persamaan matematika di atas, maka agent akan terus mengambil <i>action</i> yang sama pada tiap <i>episode</i> latihan yang membuat <i>Q-value</i> sebesar-besarnya tanpa mengeksplorasi hal baru. Padahal <b>terdapat kemungkinan bahwa solusi terbaik adalah solusi yang belum dicoba</b>. Untuk mengatasi hal tersebut, kita harus membuat parameter lain yaitu <b>$\epsilon$</b> sehingga <b>bila <i>reward</i> yang didapat kurang dari atau sama dengan $\epsilon$</b>, kita buat agent tersebut <b>memilih <i>action</i> secara random</b>. 

Itulah penjelasan tentang <i>Reinforcement Learning</i>, <i>Q-learning</i>, dan beberapa prosedurnya. Dapat dilihat bahwa <i>Q-learning</i> tidak menggunakan model apapun dan hanya berfokus pada <i>action</i> dan <i>reward</i> sehingga disebut juga <i>model-free</i>. Perlu diperhatikan bahwa <b>bila kemungkinan <i>state</i> serta <i>action</i> yang ada sangat banyak</b> maka kelemahan Q-Learning adalah harus <b>mendaftar satu persatu <i>Q-value</i></b> tersebut. Pastinya ini akan memakan banyak memori dan waktu setiap kali kita mengeksplorasi <i>state</i> dan <i>action</i> baru.
</div>





<a id = "deepqlearning"></a>
## Prinsip Deep Q-Learning

<div style = "text-align: justify ; line-height: 1.75 em">Telah kita ketahui bahwa salah satu kelemahan <i>Q-learning</i> adalah memori dan waktu karena harus mempelajari dan menyimpan <i>Q-value</i> dari setiap <i>state</i> dan <i>action</i>. Bila kita batasi proses pembelajaran agent tersebut untuk menghemat waktu dan memori, maka tidak semua kemungkinan dicoba. Saat agent tersebut selesai belajar dan dijalankan terdapat kemungkinan agent tersebut menemukan <i>state</i> baru. Akibatnya agent tersebut tidak tahu harus berbuat apa. Dengan kata lain, kekurangan <i>Q-learning</i> adalah tidak melakukan generalisasi terhadap <i>state</i> dan <i>action</i> yang mungkin. 
<br>
<br>
Namun bagaimana bila kita bisa mengestimasi berbagia kemungkinan <i>Q-value</i>? Itulah yang dilakukan oleh <i>Deep Q-learning</i>. <i>Deep Q-learning</i> berusaha mengestimasi <i>Q-value</i> dari <i>action</i> yang diambil untuk setiap <i>state</i> yang ada. Input dari <i>Deep Q-learning</i> adalah gambar <i>state</i> saat ini.

<center><img src = "https://pic4.zhimg.com/80/v2-67ef75bb7f5e67b2a42645aa821894bf_hd.png"></center>
<center>Ilustrasi Deep Q-learning pada game Atari (sumber <a src = "https://zhuanlan.zhihu.com/p/25239682">diambil di sini</a>)</center>
<br>
<b>Target dari proses <i>training neural network</i> ini adalah <i>immadiate reward</i> + ($\gamma$ x <i>estimate of future value</i>)</b> atau suku yang dinamakan <i>learned value</i> pada persamaan <i>update Q-value</i> dan <b><i>Loss function</i></b> yang ingin diminalkan adalah <b>(<i>Q-value</i> prediksi - <i>Q-value</i> sebenarnya)<sup>2</sup></b>. Tapi masalahnya adalah, kita juga tidak tau berapa <i>Q-value</i> sebenarnya saat mengambil <i>action</i> tertentu di <i>state</i> tertentu. Jadi kita harus memprediksi kedua nilai <i>Q-value</i> tersebut sehingga tidak mungkin digunakan satu neural network saja.
<br>
<br>
Untuk mengatasi masalah di atas, maka <b>satu input akan dimasukkan ke dua <i>neural network</i> yang berbeda</b>. Satu <i>neural network</i> berfungsi untuk mengestimasi target, dan <i>neural network</i> lainnya digunakan untuk memprediksi <i>Q-value</i> hasil prediksi. Lalu hasilnya training keduanya dimasukkan ke dalam <i>cost function</i>. Namun, <i>weight</i> yang selalu diupdate hanya <i>weight</i> dari <i>neural network</i> kedua sementara <i>weight</i> dari <i>neural network</i> pertama dibuat semi-konstan. Dengan kata lain, <i>weight neural network</i> pertama diperbarui dengan nilai <i>weight neural network</i> kedua hanya setiap beberapa iterasi sekali. Ini dilakukan terus menerus hingga <i>cost function</i> mencapai batas <i>threshold</i> minimal. Arsitektur dari <i>Deep Q-learning</i> dapat dilihat seperti gambar di bawah ini.
<img src = "https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/04/Screenshot-2019-04-17-at-12.48.05-PM-768x638.png" height = "500" width = "500">
<br>
Setalah mengetahui arsitekturnya, kita juga harus menentukan bagaimana cara <i>trainingnya</i> agar tidak menghabiskan memori dan waktu untuk melakukan training <i>neural network</i> sepanjang waktu. <b>Cara training</b> yang digunakan adalah <b><i>Experience Replay</i></b>. Jadi pertama kita tentukan berapa banyak <i>batch</i> yang akan dimasukkan ke <i>neural network</i> untuk di<i>train</i>. Kemudian jalankan terlebih dahulu agent untuk mengambil <i>Q-value</i> pada beberapa <i>state</i> dan <i>action</i> dan simpan dalam sebuah memori. Bila <i>Q-value</i> untuk setiap transisi <i>state</i> telah mencapai nilai <i>batch</i>, maka ambil data-data tersebut dan <i>train</i> di <i>neural network</i> dengan arsitektur seperti di atas. Selanjutnya jalankan kembali agent. Setiap terdapat data baru sejumlah <i>batch</i>, maka kita lakukan sampling sebanyak <i>batch</i> dan lakukan <i>training</i> di <i>neural network</i>. Keseluruhan workflow <i>Deep Q-learning</i> dapat dilihat di bawah (diambil dari <a src = "https://towardsdatascience.com/introduction-to-various-reinforcement-learning-algorithms-i-q-learning-sarsa-dqn-ddpg-72a5e0cb6287">artikel ini</a>).
<img src = "https://miro.medium.com/max/1508/1*nb61CxDTTAWR1EJnbCl1cA.png" height = "500" width = "500">
</div>


<a id = "implementasi"></a>

## Implementasi untuk ConnectX

Secara umum, tahap implementasi ini terdiri dari :
1. Tahap Persiapan : mempersiapkan *environment* ConnectX, *neural network* untuk proses pelatihan (*training*), dan agent
2. Tahap Pelatihan Agent : mengaktifkan *environment*, mengatur *hyperparameter*, memulai pelatihan, dan melihat hasil pelatihan 
3. Menuliskan Agent ke Sebuah File 
4. Melihat Hasil Agent

<a id = "persiapan"></a>

## Tahap Persiapan

Panduan untuk tahap ini bisa dilihat di https://www.kaggle.com/ajeffries/connectx-getting-started

In [ ]:
#  ConnectX hanya bisa dijalankan pada kaggle versi 0.1.6

!pip install 'kaggle-environments>=0.1.6' > /dev/null 2>&1

In [ ]:
# Import modul yang diperlukan

# Modul umum
import  numpy as np                # perhitungan matematika
from tqdm.notebook import tqdm     # menampilkan progress bar
import matplotlib.pyplot as plt    # menggambar plot

# Modul mempersiapkan environment ConnectX
import gym
from kaggle_environments import evaluate, make

# Modul neural network
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Mendefinisikan class yang akan dipakai sepanjang kernel ini

class ConnectX(gym.Env) :
    
    # constructor method (inisialisasi)
    def __init__(self, switch_prob = 0.5) :
        
        # membuat environment (method-methodnya dapat dilihat pada kernel Getting Started di atas)
        self.env = make('connectx', debug = False)
        # mengambil setting pada environment ConnectX (berisi timeout, columns, rows, inarow atau syarat berhasil, dan steps)   
        config = self.env.configuration
        # mendefinisikan jumlah aksi yang dapat dilakukan (mengisi kolom ke berapa) atau action space dan jumlah state atau obs space
        self.action_space = gym.spaces.Discrete(config.columns)
        self.observation_space = gym.spaces.Discrete(config.columns * config.rows)
        
        # melatih agent kita sebagai player pertama melawan agent Negamax
        # memasukkan parameter [None, "negamax"] ke dalam atribut sendiri agar mudah diakses dan diganti (misal bertukar posisi player) 
        self.pair = [None, 'negamax']     
        self.trainer = self.env.train(self.pair)
        # mendefinisikan atribut peluang untuk berganti posisi player dalam proses pelatihan agent nantinya
        self.switch_prob = switch_prob
        
    # method untuk berganti posisi player    
    def switch_trainer(self) :   
        # mengganti urutan elemen pada self.pair yang telah diinisiasi
        self.pair = self.pair[::-1]
        self.trainer = self.env.train(self.pair)     
    def reset(self) :
        if np.random.random() < self.switch_prob :
            self.switch_trainer()
        return self.trainer.reset()
    
    # method untuk mengakses observasi, reward, status done atau belum, dan info selama pelatihan agent sesuai dengan action yang dilakukan
    def step(self, action) :
        return self.trainer.step(action)
    
    # method untuk merender state
    def render(self, **kwargs) :
        return self.env.render(**kwargs)

# Mendefinisikan kelas untuk neural network untuk proses pelatihan agent (lihat https://pytorch.org/docs/stable/nn.html#containers)

class DeepModel(nn.Module) :
    
    # constructor method (inisialisai)
    def __init__(self, num_states, hidden_units, num_actions) :
         
        super(DeepModel, self).__init__()
        
        # mengkonstruksi hidden layer (perhatikan ilustrasi struktur neural network pada gambar di atas)
        # akan dicoba activation function berupa ReLU 
        self.hidden_layers = []
        for i in range(len(hidden_units)) :
            # untuk hidden layer pertama
            if i == 0 :
                self.hidden_layers.append(nn.Linear(num_states, hidden_units[i]))
            # untuk hidden layer berikutnya
            else :
                self.hidden_layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
        self.output_layer = nn.Linear(hidden_units[-1], num_actions)
    
    def forward(self, x) :
        for layer in self.hidden_layers :
            x = layer(x).clamp(min=0)
        x = self.output_layer(x)
        return x
    
# Mendefinisikan kelas untuk agent
# Secara umum, agent harus punya 3 kemampuan : bermain beberapa permainan, mengingatnya, dan memperkirakan reward dari tiap state
# yang muncul pada permainan

class DQN :

    # constructor method (inisialisasi)
    def __init__(self, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr) :
        
        #inisialisasi atribut untuk bermain
        self.num_actions = num_actions   # banyaknya aksi
        self.gamma = gamma               # porsi future reward terhadap immadiate reward
        
        # inisialisasi atribut untuk mekanisme mengingat permainan
        self.batch_size = batch_size
        self.experience = {'s' : [], 'a' : [], 'r' : [], 's2' : [], 'done' : []}
        self.max_experiences = max_experiences
        self.min_experiences = min_experiences
        
        # inisialisasi atribut untuk perkiraan reward dengan neural network di setiap state
        self.model = DeepModel(num_states, hidden_units, num_actions)
        self.optimizer = optim.Adam(self.model.parameters(), lr = lr)
        self.criterion = nn.MSELoss()
        
    # fungsi untuk mengatur permainan yang diingat lalu melakukan training neural network dari sana
    
    # membuang ingatan state paling awal bila sudah melebihi batas memori
    def add_experience(self, exp) :
        if len(self.experience['s']) >= self.max_experiences :
            for key in self.experience.keys() :
                self.experience[key].pop(0)
        for key, value in exp.items() :
            self.experience[key].append(value)
    
    # melakukan training dari neural network 
    # panduannya bisa dilihat di https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
    def train(self, TargetNet) :
        
        # hanya melakukan training bila state yang diingat telah melebihi batas minimal
        if len(self.experience['s']) < self.min_experiences :
            return 0
        
        # mengambil ingatan permainan secara random sesuai ukuran batch   
        ids = np.random.randint(low = 0, high = len(self.experience['s']), size = self.batch_size)
        states = np.asarray([self.preprocess(self.experience['s'][i]) for i in ids])
        actions = np.asarray([self.experience['a'][i] for i in ids])
        rewards = np.asarray([self.experience['r'][i] for i in ids])
        
        # mengambil label
        next_states = np.asarray([self.preprocess(self.experience['s2'][i]) for i in ids])
        dones = np.asarray([self.experience['done'][i] for i in ids])
                
        # untuk semua ingatan yang diambil, kita harus memprediksi maksimal Q-value dari state setelahnya
        value_next = np.max(TargetNet.predict(next_states).detach().numpy(), axis = 1)
        actual_values = np.where(dones, rewards, rewards + self.gamma * value_next)
        
        actions = np.expand_dims(actions, axis = 1)
        actions_one_hot = torch.FloatTensor(self.batch_size, self.num_actions).zero_()
        actions_one_hot = actions_one_hot.scatter_(1, torch.LongTensor(actions), 1)
        selected_action_values = torch.sum(self.predict(states) * actions_one_hot, dim = 1)
        actual_values = torch.FloatTensor(actual_values)
        
        self.optimizer.zero_grad()
        loss = self.criterion(selected_action_values, actual_values)
        loss.backward()
        self.optimizer.step()
    
    def copy_weights(self, TrainNet) :
        self.model.load_state_dict(TrainNet.state_dict())
        
    def save_weights(self, path) :
        torch.save(self.model.state_dict(), path)
        
    def load_weights(self, path) :
        self.model.load_state_dict(torch.load(path))      
        
    # fungsi untuk menentukan aksi yang diambil untuk bermain berdasarkan hasil prediksi dan aturan eksplorasi
    
    # fungsi untuk preprocess state sebelum dimas
    def preprocess(self, state) :
        result = state.board[:]
        result.append(state.mark)
        return result
    
    # fungsi untuk prediksi 
    def predict(self, inputs) :
        return self.model(torch.from_numpy(inputs).float())
    
    def get_action(self, state, epsilon) :
        # mekanisme eksplorasi, melakukan aksi random bila kolom tersebut kosong 
        if np.random.random() < epsilon :
            return int(np.random.choice([c for c in range(self.num_actions) if state.board[c] == 0]))
        else :
            prediction = self.predict(np.atleast_2d(self.preprocess(state)))[0].detach().numpy()
            for i in range(self.num_actions) :
                if state.board[i] != 0 :
                    prediction[i] = -1e7
            # melakukan aksi dengan menaruh checker di kolom yang punya hasil prediksi Q value yang paling besar
            return int(np.argmax(prediction)) 

# mendefinisikan aturan permainan yang akan dipelajari agent
def play_game(env, TrainNet, TargetNet, epsilon, copy_step) :
    rewards = 0
    iter = 0
    done = False
    observations = env.reset()
    
    while not done :     
        action = TrainNet.get_action(observations, epsilon)
        prev_observations = observations
        observations, reward, done, _ = env.step(action)  
        if done :
            # menang
            if reward == 1 :
                reward = 20
            # kalah
            elif reward == 0 :
                reward = -50
            # draw
            else :
                reward = 10
        # membuat agent kita berusaha untuk bermain lebih panjang (selama permainan belum berakhir dia mendapat reward 0.5)
        # namun harus dibuat threshold agar bila telah melewati batas tertentu maka agen berusaha menang (bukan berusaha main lebih panjang)    
        else :
            if rewards <= 2.5 : 
                reward = 0.5
            else :
                reward = -0.5
        rewards += reward
        
        # membuat buffer ingatan permainan
        exp = {'s' : prev_observations, 'a' : action, 'r' : reward, 's2' : observations, 'done' : done}
        TrainNet.add_experience(exp)
        TrainNet.train(TargetNet)
        
        # ingat bahwa kita membuat network yang memprediksi nilai sebenarnya dari target akan mengcopy weight dari network satu lagi
        iter+=1
        if iter % copy_step == 0 :
            TargetNet.copy_weights(TrainNet)
            
    return rewards    

<a id = "pelatihan"></a>

## Tahap Pelatihan Agent

In [ ]:
# mengaktifkan environment

env = ConnectX()

# mengatur hyperparameter

gamma = 0.99 
copy_step = 25
hidden_units = [100, 200, 200, 100]
max_experiences = 1000
min_experiences = 100
batch_size = 32
lr = 0.01
epsilon = 0.25    # lebih memilih eksploitasi, namun decay epsilon diperlambat
decay = 0.99
min_epsilon = 0.05
episodes =14000
precision = 7

In [ ]:
# inisiasi

num_states = env.observation_space.n + 1
num_actions = env.action_space.n

all_total_rewards = np.empty(episodes) 
all_avg_rewards = np.empty(episodes)
all_epsilons = np.empty(episodes)

TrainNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
TargetNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)

progress_bar = tqdm(range(episodes))
for i in progress_bar :
    if i % 10 == 0 :
        epsilon = max(min_epsilon, epsilon * decay)
    else :
        epsilon = max(min_epsilon, epsilon)
    total_reward = play_game(env, TrainNet, TargetNet, epsilon, copy_step)
    all_total_rewards[i] = total_reward
    avg_reward = all_total_rewards[max(0, i-100) : (i+1)].mean()
    all_avg_rewards[i] = avg_reward
    all_epsilons[i] = epsilon
    progress_bar.set_postfix({
        'episode_reward' : total_reward,
        'avg of last 100 reward' : avg_reward,
        'epsilon' : epsilon
    })

In [ ]:
# melihat hasil pelatihan agent

plt.plot(all_avg_rewards)
plt.xlabel('Episode')
plt.ylabel('Avg Last 100 Rewards ')
plt.show()

# menyimpan weight dari tiap state

TrainNet.save_weights('./weights.pth')

<a id = "menulis"></a>

## Menuliskan Agent ke Sebuah File

In [ ]:
fc_layers = []

# mengambil weight dan bias dari tiap hidden layer serta output layer
for i in range(len(hidden_units)):
    fc_layers.extend([
        TrainNet.model.hidden_layers[i].weight.T.tolist(), 
        TrainNet.model.hidden_layers[i].bias.tolist() 
    ])
fc_layers.extend([
    TrainNet.model.output_layer.weight.T.tolist(), 
    TrainNet.model.output_layer.bias.tolist() 
])

# mengedit hasil dari fc_layers
fc_layers = list(map(
    lambda x: str(list(np.round(x, precision))) \
        .replace('array(', '').replace(')', '') \
        .replace(' ', '') \
        .replace('\n', ''),
    fc_layers
))
fc_layers = np.reshape(fc_layers, (-1, 2))

# Menuliskan agent
# Agent tersebut harus diinisiasi terlebih dahulu, mempunyai daftar weights tiap state untuk melakukan perhitungan, dan melakukan aksi

# inisiasi agent
my_agent = '''def my_agent(observation, configuration):
    import numpy as np

'''

# memasukkan hasil bobot tiap hidden layer
for i, (w, b) in enumerate(fc_layers[:-1]):
    my_agent += '    hl{}_w = np.array({}, dtype=np.float32)\n'.format(i+1, w)
    my_agent += '    hl{}_b = np.array({}, dtype=np.float32)\n'.format(i+1, b)
my_agent += '    ol_w = np.array({}, dtype=np.float32)\n'.format(fc_layers[-1][0])
my_agent += '    ol_b = np.array({}, dtype=np.float32)\n'.format(fc_layers[-1][1])

my_agent += '''
    state = observation.board[:]
    state.append(observation.mark)
    out = np.array(state, dtype=np.float32)

'''

# melakukan kalkulasi Q-value berdasarkan weight hidden layer hingga output layer
for i in range(len(fc_layers[:-1])):
    my_agent += '    out = np.matmul(out, hl{0}_w) + hl{0}_b\n'.format(i+1)
    my_agent += '    out = np.maximum(out,0)\n'     # fungsi aktivasi ReLU .clamp(min = 0)
    
my_agent += '    out = np.matmul(out, ol_w) + ol_b\n'

# melakukan aksi
my_agent += '''
    for i in range(configuration.columns):
        if observation.board[i] != 0:
            out[i] = -1e7

    return int(np.argmax(out))
    '''
with open('submission.py', 'w') as f:
    f.write(my_agent)

<a id = "hasil"></a>

## Melihat Hasil Agent

In [ ]:
from submission import my_agent

def mean_reward(rewards):
    return sum(r[0] for r in rewards) / sum(r[0] + r[1] for r in rewards)

print("My Agent vs. Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=50)))
print("My Agent vs. Negamax Agent:", mean_reward(evaluate("connectx", [my_agent, "negamax"], num_episodes=50)))
print("Random Agent vs. My Agent:", mean_reward(evaluate("connectx", ["random", my_agent], num_episodes=50)))
print("Negamax Agent vs. My Agent:", mean_reward(evaluate("connectx", ["negamax", my_agent], num_episodes=50)))